In [ ]:
import re
import json
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import time

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

text_data = ""

folder_path = '/content/drive/MyDrive/Books/ZLN(2)11.2.25'

def visit_files_in_directory(root_dir):
    global text_data
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            try:
                with open(filepath, "r", encoding="utf-8") as f:
                    text_data += f.read()
            except Exception as e:
                print(f"Error reading file {filepath}: {e}")

# Call the function to visit all files
visit_files_in_directory(folder_path)

# Print the results
print("Total number of characters:", len(text_data))
print(text_data[:99])  # Print first 99 characters

Total number of characters: 7459593
﻿
 
ပထမံ ဆရာသိန်း
ရွှေဥဒေါင်း
ပုံနှိပ်မှတ်တမ်း
 
စာမူခွင့်ပြုချက်အမှတ် [၃၇၇ / ၉၆(၇)]
မျက်နှာဖုံးခွင


In [ ]:
def create_break_pattern():
    """Creates and returns the regular expression pattern for Myanmar syllable breaking."""
    my_consonant = r"က-အ၎"
    en_char = r"a-zA-Z0-9"
    other_char = r"ဣဤဥဦဧဩဪဿ၌၍၏၀-၉၊။!-/:-@[-`{-~\s“”—‘’…><"
    subscript_symbol = r'္'
    a_that = r'်'

    # Regular expression pattern for Myanmar syllable breaking
    return re.compile(
        r"((?<!" + subscript_symbol + r")[" + my_consonant + r"]"
        r"(?!["
        + a_that + subscript_symbol + r"])"
        + r"|[" + en_char + other_char + r"])"
    )

def process_string(input_string):
    # Function to check if the character is valid (contains '်', '့' and exactly one consonant)
    def is_valid_char(char):
        return '်' in char and '့' in char and len(char) == 3

    # Function to rearrange the Unicode order of specific marks
    def rearrange_unicode_order(word):
        splited = [c for c in word]
        splited[1], splited[2] = splited[2], splited[1]
        return ("").join(splited)

    # Process the string
    updated_string = []
    for word in input_string.split():
        if is_valid_char(word) and updated_string:
            updated_string[-1] += rearrange_unicode_order(word)  # Merge with the previous word
        else:
            updated_string.append(word)  # Otherwise, append as a new word

    return ' '.join(updated_string)

# main function to use sylbreak.py
def break_syllables(line):
    """Applies syllable breaking rules to a line."""
    break_pattern = create_break_pattern()
    separator = ' '
    line = re.sub(r'\s+', ' ', line.strip())
    segmented_line = break_pattern.sub(separator + r"\1", line)

    # Remove the leading delimiter if it exists
    if segmented_line.startswith(separator):
        segmented_line = segmented_line[len(separator):]

    # Replace delimiter+space+delimiter with a single space
    double_delimiter = separator + " " + separator
    segmented_line = segmented_line.replace(double_delimiter, " ")

    segmented_line = process_string(segmented_line)

    return segmented_line



test_text = "ကြည့်ပါ"
tokenized_text = break_syllables(test_text)
print(tokenized_text)

ကြည့် ပါ


In [ ]:
def load_vocab_from_json(file_path):
    with open(file_path, 'r') as f:
        vocab = json.load(f)
    return vocab

vocab_dict = load_vocab_from_json("/content/drive/MyDrive/vocabs.json")
print(len(vocab_dict))

3631


In [ ]:
GPT_CONFIG = {
    "vocab_size": 3631,   # Vocabulary size
    "context_length": 128, # Shortened context length (orig: 1024)
    "emb_dim": 512,        # Embedding dimension
    "n_heads": 8,         # Number of attention heads
    "n_layers": 8,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}


In [ ]:
def text_to_token_ids(text):
    # Step 1: Replace spaces with "<_>"
    text_with_tokens = text.replace(" ", "<_>")
    unknowns = []

    # Step 2: Break into syllables while keeping "<_>" intact
    syllables_string = break_syllables(text_with_tokens)

    # Step 3: Split into syllables (keeping "<_>" as a unit)
    syllables_list = syllables_string.split(" ")

    # Step 4: Convert syllables to token IDs
    encoded_ids = []
    i = 0
    while i < len(syllables_list):
        syllable = syllables_list[i]

        # Detect "<_>" token pattern
        if syllable == "<" and i + 2 < len(syllables_list) and syllables_list[i + 1] == "_" and syllables_list[i + 2] == ">":
            encoded_ids.append(vocab_dict["<_>"])
            i += 3  # Skip '<', '_', and '>'
        elif syllable in vocab_dict:
            encoded_ids.append(vocab_dict[syllable])
            i += 1
        else:
            unknowns.append(syllable)
            encoded_ids.append(vocab_dict.get("<UNK>", 0))  # Default to 0 if "<UNK>" isn't in vocab
            i += 1

    # Step 5: Convert to tensor with batch dimension
    print(set(unknowns))
    print(len(unknowns))
    print(len(set(unknowns)))
    return torch.tensor(encoded_ids).unsqueeze(0)


def token_ids_to_text(token_ids):
    # Step 1: Remove batch dimension
    flat_ids = token_ids.squeeze(0).tolist()  # Convert tensor to list

    # Step 2: Create a reverse mapping (ID -> word)
    reverse_vocab_dict = {v: k for k, v in vocab_dict.items()}

    # Step 3: Find the ID for "<_>" (space token)
    space_id = vocab_dict.get("<_>", None)

    # Step 4: Decode each token ID
    decoded_words = []
    for token_id in flat_ids:
        if token_id == space_id:
            decoded_words.append(" ")  # Replace "<_>" with space
        elif token_id in reverse_vocab_dict:
            decoded_words.append(reverse_vocab_dict[token_id])  # Convert ID to syllable
        else:
            decoded_words.append("<UNK>")  # Handle unknown tokens

    # Step 5: Join the decoded words into a string
    return "".join(decoded_words)  # Keep spacing intact

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), ## Expansion
            GELU(), ## Activation
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]), ## Contraction
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        # 2*4*768
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x
        # 2*4*768

In [ ]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

model = GPTModel(GPT_CONFIG)
model.eval();  # Disable dropout during inference

In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [ ]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context

    ###Input batch:
 ###tensor([[6109, 3626, 6100,  345],
        ##[6109, 1110, 6622,  257]])

    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond) ### batch, n_tokens, vocab_size

        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [ ]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

In [ ]:
def generate_and_print_sample(model, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
    decoded_text = token_ids_to_text(token_ids)
    print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()

In [ ]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            tokens_seen += input_batch.numel() # Returns the total number of elements (or tokens) in the input_batch.
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)

            if global_step % 25 == 0:
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen

<div class="alert alert-block alert-success">
Pre-train
</div>

In [ ]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = text_to_token_ids(txt)
        token_ids = token_ids.squeeze(0)

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(input_chunk.clone().detach())
            self.target_ids.append(target_chunk.clone().detach())

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Create dataset
    dataset = GPTDatasetV1(txt, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [ ]:
# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG["context_length"],
    stride=GPT_CONFIG["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG["context_length"],
    stride=GPT_CONFIG["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

{'နူတ်', 'ကုန္ဒ', 'ဗျုး', 'တ\u200c', 'ဝစ္ဆေ', 'ဇိင်း', 'တော့ည်', 'ခံ\ufeff', 'ပြဟ်', 'ဒူလ်', 'တ့', 'ကိုး\ufeff', 'ဇွပ်', '၀ူး', 'လိင်း', 'ဒစ္စ', 'ယိန်', 'မို့စ်', 'မြှင်', 'ဆောတ်', 'ဇာ့တ်', 'နေ\ufeff', 'ဟောလ်', 'ဂ္ဂိုင်', 'ကျွန့်', 'ချုန်', 'ဂျင်း', 'နက်ဖ်', 'တဲ့\ufeff', 'ဗွိုင်း', 'ကြိတ်\ufeff', 'နား\ufeff', 'လျာက်', 'တိက်္က', 'ဒီး\ufeff', 'ဝတ်\ufeff', 'စောင့်င့်', 'ကုဏ္ဍ', 'ဿာန်', 'ခန္ဒာ', 'nç', 'လို့\ufeff', '၀ုိင်း', '>ဋ္ဌာ', 'ဟုမ်းစ်', 'နည်း\ufeff', 'ဟာ\u200c', 'လျှောင်', 'မြွင့်', 'ဝန်\ufeff', 'နွိုက်', '၊ည်', 'ရက်စ်', 'ဖွင်', 'နာည်း', 'ဗြန်း', '-ိလ်', 'ဥ\u200c\u200c', 'စာင့်', 'နဲ့\ufeff', 'မီ\ufeff', 'ညီ\u200c', 'အိမ်\ufeff', 'မာ\u200c', 'ဝ\u200c', 'ဒက္ကာ', 'သန်္နိဋ္ဌာန်', 'ရှိန့်', 'ဘော့ဗ်', 'ဇီစ်', 'ကူ့', 'ဝုဖ်', 'ရွှီးတ်', 'ရေွး', 'ရစ်ခ်ျ', 'လောက်\ufeff', 'အို့တ်', 'ဆော့ဖ်', 'အောက်စ်', 'ဟတ်ခ်', 'မြန်\ufeff', 'ဆေ', 'လင်္ဂ', 'ပေါတ်', 'သမ်္ဗာန်', 'ကျာ်', 'ဒမ်', 'ဆွေး\u200c', '၀ှက်', 'ဒွတ္တ', 'မစ္စက်', 'တော့\ufeff', 'ဂွတ်', 'ရှုး', 'မား\ufeff', 'သန္ဒိ', 'ဆုံုး', 'ဘီးလ်', 'ဤတ်', 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 8.36904888132595
Validation loss: 8.367079822629767


In [ ]:
start_time = time.time()

model = GPTModel(GPT_CONFIG)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 4
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="ဘာ",
)

# Note:
# Uncomment the following code to show the execution time
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 7.195, Val loss 7.366
Ep 1 (Step 000025): Train loss 5.413, Val loss 5.650
Ep 1 (Step 000050): Train loss 5.075, Val loss 5.381
Ep 1 (Step 000075): Train loss 5.141, Val loss 5.243
Ep 1 (Step 000100): Train loss 4.819, Val loss 5.120
Ep 1 (Step 000125): Train loss 4.919, Val loss 5.037
Ep 1 (Step 000150): Train loss 4.670, Val loss 5.043
Ep 1 (Step 000175): Train loss 4.735, Val loss 5.001
Ep 1 (Step 000200): Train loss 4.488, Val loss 4.951
Ep 1 (Step 000225): Train loss 4.614, Val loss 4.915
Ep 1 (Step 000250): Train loss 4.600, Val loss 4.906
Ep 1 (Step 000275): Train loss 4.599, Val loss 4.834
Ep 1 (Step 000300): Train loss 4.392, Val loss 4.819
Ep 1 (Step 000325): Train loss 4.379, Val loss 4.838
Ep 1 (Step 000350): Train loss 4.476, Val loss 4.847
Ep 1 (Step 000375): Train loss 4.268, Val loss 4.826
Ep 1 (Step 000400): Train loss 4.370, Val loss 4.747
Ep 1 (Step 000425): Train loss 4.068, Val loss 4.709
Ep 1 (Step 000450): Train loss 4.383, Val loss

In [ ]:
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):

    # For-loop is the same as before: Get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float("-inf")).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next == eos_id:  # Stop generating early if end-of-sequence token is encountered and eos_id is specified
            break

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx

In [ ]:
# Save the model's state_dict
torch.save(model.state_dict(), "model.pth")
print("Model saved successfully!")

Model saved successfully!


In [ ]:
# Initialize your model (must match the saved model architecture)
model = YourModel()  # Replace with your actual model class

# Load state_dict into model
model.load_state_dict(torch.load("model.pth"))

# Set model to evaluation mode (important for inference)
model.eval()

print("Model loaded successfully!")

NameError: name 'YourModel' is not defined

In [ ]:
# Generate token IDs
token_ids = generate(
    model=model,
    idx=text_to_token_ids("ဟုတ်").to(device),  # Move idx to the correct device
    max_new_tokens=30,
    context_size=GPT_CONFIG["context_length"],
    top_k=25,
    temperature=1.4
)

print("Output text:\n", token_ids_to_text(token_ids))

Output text:
 ဟုတ်သည် ထိုစကား တစ်ရပ် ပြောနေဟန် ရှိ၏ ။သူသည် ရုတ်ချည်းပင်<UNK><UNK><UNK>ငါကလို


In [ ]:
model.eval() #A
#model = GPTModel(GPT_CONFIG)
out = generate_text_simple(
model=model,
idx=encoded_tensor,
max_new_tokens=6,
context_size=GPT_CONFIG["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))

NameError: name 'encoded_tensor' is not defined